In [ ]:
import os
from tqdm import tqdm
#from tensorflow.python.compiler.mlcompute import mlcompute
import tensorflow as tf
import numpy as np
import pandas as pd
import librosa

import time
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt

# Train Soundscapes Dataset

In [ ]:
train_soundscape = pd.read_csv('../input/birdclef-2021/train_soundscape_labels.csv',)
train_soundscape

In [ ]:
print("Anzahl der Datensätze: %s" % (
    train_soundscape['audio_id'].value_counts().count()
))
print("Anzahl der Audio-Files: %s" % (
    train_soundscape['audio_id'].count()
))
print("Anzahl der Annotierungen: %s" % (
    sum(map(len, train_soundscape['birds'].str.split(' ',1)))
))
print("Durchschnittliche Anzahl der Annotierungen pro Audio-File: %s" % (
    train_soundscape['audio_id'].value_counts().mean()
))

#train_soundscape.groupby(['audio_id', 'birds'])['birds'].value_counts() #.apply(display)
fig, axs = plt.subplots(figsize=(12, 4))
train_soundscape['birds'].str.split(' ',1).str.len().value_counts().plot(
    kind='bar', rot=0, ax=axs, legend=False,
    title='Annotierungen pro Datensatz', 
    xlabel='Anzahl Annotierungen',
    ylabel='Datensätze',
    color='black',
)

In [ ]:
# Anzahl der Datensätze für Call / Nocall
fig, axs = plt.subplots(figsize=(12, 4))

train_soundscape['call_nocall'] = train_soundscape['birds']
train_soundscape['call_nocall'][train_soundscape['birds'] != 'nocall'] = 'call'

train_soundscape['call_nocall'].value_counts().plot(
    kind='bar', rot=0, ax=axs, legend=False,
    title='Anzahl der Datensätze für Call / Nocall', 
    xlabel='Anzahl Annotierungen',
    ylabel='Datensätze',
    color='black',
)

In [ ]:
fig, axs = plt.subplots(figsize=(12, 4))

train_soundscape['birds'].str.split(' ',1).explode().value_counts().drop(labels="nocall", axis=0).plot(
    kind='bar', rot=0, ax=axs, legend=False,
    title='Verteilung Anzahl der Datensätze pro Art', 
    xlabel='Art',
    ylabel='Anzahl Datensätze',
    color='black',
)
axs.axes.xaxis.set_ticks([]) # remove Labels for X

# Training Dataset

In [ ]:
train = pd.read_csv('../input/birdclef-2021/train_metadata.csv',)
train

In [ ]:
# Lade den Deutschen Namen von https://ebird.org/species/acafly
import requests

def get_german_name(primary_label):
    payload = {
        'fmt': 'json',
        'locale': 'de',
        'species': primary_label
    }
    r = requests.get('https://api.ebird.org/v2/ref/taxonomy/ebird',
        params=payload,
        headers={
            'X-eBirdApiToken': 'RXpK1BNzbJNcHfBs'
        }
    )
    data = r.json()
    if data[0] and 'comName' in data[0]:
        return data[0]['comName']
    return 'nicht bekannt'

# sollte nicht jedes mal angefragt werden
# german_names = {}
# for primary_label in train.primary_label.unique():
#    german_names[primary_label] = get_german_name(primary_label)

german_names = {'acafly': 'Buchenschnäppertyrann', 'acowoo': 'Eichelspecht', 'aldfly': 'Erlenschnäppertyrann', 'ameavo': 'Braunhals-Säbelschnäbler', 'amecro': 'Amerikakrähe', 'amegfi': 'Goldzeisig', 'amekes': 'Buntfalke', 'amepip': 'Pazifikpieper', 'amered': 'Rotschwanz-Waldsänger', 'amerob': 'Wanderdrossel', 'amewig': 'Kanadapfeifente', 'amtspa': 'Baumammer', 'andsol1': 'Andenklarino', 'annhum': 'Annakolibri', 'astfly': 'Kalifornienschopftyrann', 'azaspi1': 'Azaradickichtschlüpfer', 'babwar': 'Braunkehl-Waldsänger', 'baleag': 'Weißkopf-Seeadler', 'balori': 'Baltimoretrupial', 'banana': 'Zuckervogel', 'banswa': 'Uferschwalbe', 'banwre1': 'Akazienzaunkönig', 'barant1': 'Bindenameisenwürger', 'barswa': 'Rauchschwalbe', 'batpig1': 'Bandtaube', 'bawswa1': 'Schwarzsteißschwalbe', 'bawwar': 'Kletterwaldsänger', 'baywre1': 'Kastanienzaunkönig', 'bbwduc': 'Rotschnabel-Pfeifgans', 'bcnher': 'Nachtreiher', 'belkin1': 'Gürtelfischer', 'belvir': 'Braunaugenvireo', 'bewwre': 'Buschzaunkönig', 'bkbmag1': 'Hudsonelster', 'bkbplo': 'Kiebitzregenpfeifer', 'bkbwar': 'Fichtenwaldsänger', 'bkcchi': 'Schwarzkopfmeise', 'bkhgro': 'Schwarzkopf-Kernknacker', 'bkmtou1': 'Goldkehltukan', 'bknsti': 'Schwarznacken-Stelzenläufer', 'blbgra1': 'Jacarinitangare', 'blbthr1': 'Schwarzschnabeldrossel', 'blcjay1': 'Schwarzbrust-Blaurabe', 'blctan1': 'Schwarzscheiteltangare', 'blhpar1': 'Schwarzohrpapagei', 'blkpho': 'Schwarzkopf-Phoebetyrann', 'blsspa1': 'Dickichtammer', 'blugrb1': 'Azurfink', 'blujay': 'Blauhäher', 'bncfly': 'Braunschopftyrann', 'bnhcow': 'Braunkopf-Kuhstärling', 'bobfly1': 'Starkschnabel-Maskentyrann', 'bongul': 'Bonapartemöwe', 'botgra': 'Bootschwanzgrackel', 'brbmot1': 'Plattschnabelmotmot', 'brbsol1': 'Braunrückenklarino', 'brcvir1': 'Braunkappenvireo', 'brebla': 'Purpurstärling', 'brncre': 'Amerikabaumläufer', 'brnjay': 'Braunhäher', 'brnthr': 'Rotspottdrossel', 'brratt1': 'Gelbbürzel-Attilatyrann', 'brwhaw': 'Breitflügelbussard', 'brwpar1': 'Glanzflügelpapagei', 'btbwar': 'Blaurücken-Waldsänger', 'btnwar': 'Grünmantel-Waldsänger', 'btywar': 'Trauerwaldsänger', 'bucmot2': 'Diademmotmot', 'buggna': 'Blaumückenfänger', 'bugtan': 'Bischofstangare', 'buhvir': 'Graukopfvireo', 'bulori': 'Bullocktrupial', 'burwar1': 'Schmätzerwaldsänger', 'bushti': 'Buschschwanzmeise', 'butsal1': 'Buntkehlsaltator', 'buwtea': 'Blauflügelente', 'cacgoo1': 'Zwergkanadagans', 'cacwre': 'Kaktuszaunkönig', 'calqua': 'Schopfwachtel', 'caltow': 'Kaliforniengrundammer', 'cangoo': 'Kanadagans', 'canwar': 'Kanadawaldsänger', 'carchi': 'Carolinameise', 'carwre': 'Carolinazaunkönig', 'casfin': 'Cassingimpel', 'caskin': 'Cassinkönigstyrann', 'caster1': 'Raubseeschwalbe', 'casvir': 'Cassinvireo', 'categr': 'Kuhreiher', 'ccbfin': 'Braunkopf-Buschammer', 'cedwax': 'Zedernseidenschwanz', 'chbant1': 'Braunrücken-Ameisenvogel', 'chbchi': 'Rotrückenmeise', 'chbwre1': 'Kastanienbrust-Zaunkönig', 'chcant2': 'Rostkappen-Ameisenpitta', 'chispa': 'Schwirrammer', 'chswar': 'Gelbscheitel-Waldsänger', 'cinfly2': 'Zimttyrann', 'clanut': 'Kiefernhäher', 'clcrob': 'Gilbdrossel', 'cliswa': 'Fahlstirnschwalbe', 'cobtan1': 'Finkengrünammer', 'cocwoo1': 'Kakaobaumsteiger', 'cogdov': 'Sperlingstäubchen', 'colcha1': 'Kolumbienguan', 'coltro1': 'Jungferntrogon', 'comgol': 'Schellente', 'comgra': 'Purpurgrackel', 'comloo': 'Eistaucher', 'commer': 'Gänsesäger', 'compau': 'Pauraquenachtschwalbe', 'compot1': 'Klagetagschläfer', 'comrav': 'Kolkrabe', 'comyel': 'Weidengelbkehlchen', 'coohaw': 'Rundschwanzhabicht', 'cotfly1': 'Graugelb-Todityrann', 'cowscj1': 'Kalifornienhäher', 'cregua1': 'Haubenguan', 'creoro1': 'Krähenstirnvogel', 'crfpar': 'Veraguasittich', 'cubthr': 'Krummschnabel-Spottdrossel', 'daejun': 'Winterammer', 'dowwoo': 'Dunenspecht', 'ducfly': 'Schwarzkappen-Schopftyrann', 'dusfly': 'Buschland-Schnäppertyrann', 'easblu': 'Rotkehl-Hüttensänger', 'easkin': 'Schieferrücken-Königstyrann', 'easmea': 'Lerchenstärling', 'easpho': 'Weißbauch-Phoebetyrann', 'eastow': 'Rötelgrundammer', 'eawpew': 'Hellbauch-Schnäppertyrann', 'eletro': 'Kupfertrogon', 'eucdov': 'Türkentaube', 'eursta': 'Star', 'fepowl': 'Brasilzwergkauz', 'fiespa': 'Klapperammer', 'flrtan1': 'Feuerbürzeltangare', 'foxspa': 'Fuchsammer', 'gadwal': 'Schnatterente', 'gamqua': 'Helmwachtel', 'gartro1': 'Grünschwanztrogon', 'gbbgul': 'Mantelmöwe', 'gbwwre1': 'Einsiedlerzaunkönig', 'gcrwar': 'Goldhähnchen-Waldsänger', 'gilwoo': 'Gilaspecht', 'gnttow': 'Grünschwanz-Grundammer', 'gnwtea': 'Krickente', 'gocfly1': 'Andenmaskentyrann', 'gockin': 'Indianergoldhähnchen', 'gocspa': 'Kronenammer', 'goftyr1': 'Goldgesicht-Kleintyrann', 'gohque1': 'Goldkopftrogon', 'goowoo1': 'Olivmantelspecht', 'grasal1': 'Grausaltator', 'grbani': 'Riefenschnabelani', 'grbher3': 'Kanadareiher', 'grcfly': 'Gelbbauch-Schopftyrann', 'greegr': 'Silberreiher', 'grekis': 'Schwefelmaskentyrann', 'grepew': 'Mexikoschnäppertyrann', 'grethr1': 'Riesendrossel', 'gretin1': 'Großtinamu', 'greyel': 'Tüpfelgelbschenkel', 'grhcha1': 'Graukopfguan', 'grhowl': 'Virginiauhu', 'grnher': 'Grünreiher', 'grnjay': 'Inkablaurabe', 'grtgra': 'Großschwanzgrackel', 'grycat': 'Katzenspottdrossel', 'gryhaw2': 'Graubussard', 'gwfgoo': 'Blässgans', 'haiwoo': 'Haarspecht', 'heptan': 'Zinnoberkardinal', 'hergul': 'Silbermöwe', 'herthr': 'Einsiedler-Musendrossel', 'herwar': 'Einsiedelwaldsänger', 'higmot1': 'Hochlandmotmot', 'hofwoo1': 'Hoffmannspecht', 'houfin': 'Hausgimpel', 'houspa': 'Haussperling', 'houwre': 'Hauszaunkönig', 'hutvir': 'Huttonvireo', 'incdov': 'Inkatäubchen', 'indbun': 'Indigofink', 'kebtou1': 'Fischertukan', 'killde': 'Keilschwanz-Regenpfeifer', 'labwoo': 'Texasspecht', 'larspa': 'Rainammer', 'laufal1': 'Lachfalke', 'laugul': 'Aztekenmöwe', 'lazbun': 'Lazulifink', 'leafly': 'Zwergschnäppertyrann', 'leasan': 'Wiesenstrandläufer', 'lesgol': 'Mexikozeisig', 'lesgre1': 'Graukappenvireo', 'lesvio1': 'Berg-Veilchenohrkolibri', 'linspa': 'Lincolnammer', 'linwoo1': 'Linienspecht', 'littin1': 'Brauntinamu', 'lobdow': 'Tundraschlammläufer', 'lobgna5': 'Schwarzschwanz-Degenschnäbler', 'logshr': 'Louisianawürger', 'lotduc': 'Eisente', 'lotman1': 'Langschwanzpipra', 'lucwar': 'Rotbürzel-Waldsänger', 'macwar': 'Dickichtwaldsänger', 'magwar': 'Magnolienwaldsänger', 'mallar3': 'Stockente', 'marwre': 'Sumpfzaunkönig', 'mastro1': 'Maskentrogon', 'meapar': 'Mülleramazone', 'melbla1': 'Trauerstärling', 'monoro1': 'Montezumastirnvogel', 'mouchi': 'Gebirgsmeise', 'moudov': 'Carolinataube', 'mouela1': 'Nordanden-Olivtyrann', 'mouqua': 'Bergwachtel', 'mouwar': 'Graukopf-Waldsänger', 'mutswa': 'Höckerschwan', 'naswar': 'Rubinfleck-Waldsänger', 'norcar': 'Rotkardinal', 'norfli': 'Goldspecht', 'normoc': 'Gartenspottdrossel', 'norpar': 'Meisenwaldsänger', 'norsho': 'Löffelente', 'norwat': 'Drosselwaldsänger', 'nrwswa': 'Graukehlschwalbe', 'nutwoo': 'Nuttallspecht', 'oaktit': 'Eichenmeise', 'obnthr1': 'Goldschnabel-Musendrossel', 'ocbfly1': 'Ockerbauch-Pipratyrann', 'oliwoo1': 'Dünnschnabel-Baumsteiger', 'olsfly': 'Olivflanken-Schnäppertyrann', 'orbeup1': 'Gelbbauchorganist', 'orbspa1': 'Goldschnabel-Buschammer', 'orcpar': 'Tovisittich', 'orcwar': 'Orangefleck-Waldsänger', 'orfpar': 'Elfenbeinsittich', 'osprey': 'Fischadler', 'ovenbi1': 'Pieperwaldsänger', 'pabspi1': 'Weißbauch-Dickichtschlüpfer', 'paltan1': 'Palmentangare', 'palwar': 'Palmenwaldsänger', 'pasfly': 'Feuchtwald-Schnäppertyrann', 'pavpig2': 'Rotrückentaube', 'phivir': 'Philadelphiavireo', 'pibgre': 'Bindentaucher', 'pilwoo': 'Helmspecht', 'pinsis': 'Fichtenzeisig', 'pirfly1': 'Kurzschnabel-Maskentyrann', 'plawre1': 'Cabaniszaunkönig', 'plaxen1': 'Braunbauch-Baumspäher', 'plsvir': 'Weißstirnvireo', 'plupig2': 'Weintaube', 'prowar': 'Zitronenwaldsänger', 'purfin': 'Purpurgimpel', 'purgal2': 'Zwergsultanshuhn', 'putfru1': 'Purpurbrustkotinga', 'pygnut': 'Zwergkleiber', 'rawwre1': 'Rotrückenzaunkönig', 'rcatan1': 'Karmesinkardinal', 'rebnut': 'Rotbrustkleiber', 'rebsap': 'Feuerkopf-Saftlecker', 'rebwoo': 'Carolinaspecht', 'redcro': 'Fichtenkreuzschnabel', 'reevir1': 'Rotaugenvireo', 'rehbar1': 'Andenbartvogel', 'relpar': 'Rotstirnamazone', 'reshaw': 'Rotschulterbussard', 'rethaw': 'Rotschwanzbussard', 'rewbla': 'Rotflügelstärling', 'ribgul': 'Ringschnabelmöwe', 'rinkin1': 'Rotbrustfischer', 'roahaw': 'Wegebussard', 'robgro': 'Rosenbrust-Kernknacker', 'rocpig': 'Felsentaube', 'rotbec': 'Rosenkehlbekarde', 'royter1': 'Königsseeschwalbe', 'rthhum': 'Rubinkehlkolibri', 'rtlhum': 'Braunschwanzamazilie', 'ruboro1': 'Breithauben-Stirnvogel', 'rubpep1': 'Rostbrauenvireo', 'rubrob': 'Rotmanteldrossel', 'rubwre1': 'Rotbrust-Zaunkönig', 'ruckin': 'Rubingoldhähnchen', 'rucspa1': 'Morgenammer', 'rucwar': 'Rotkappen-Waldsänger', 'rucwar1': 'Goldscheitel-Waldsänger', 'rudpig': 'Purpurtaube', 'rudtur': 'Steinwälzer', 'rufhum': 'Rotrücken-Zimtelfe', 'rugdov': 'Rosttäubchen', 'rumfly1': 'Rostschwingen-Maskentyrann', 'runwre1': 'Rotnacken-Zaunkönig', 'rutjac1': 'Rotschwanz-Glanzvogel', 'saffin': 'Safrangilbtangare', 'sancra': 'Kanadakranich', 'sander': 'Sanderling', 'savspa': 'Grasammer', 'saypho': 'Zimtbauch-Phoebetyrann', 'scamac1': 'Scharlachara', 'scatan': 'Scharlachkardinal', 'scbwre1': 'Schuppenbrust-Zaunkönig', 'scptyr1': 'Rot-Schuppenkopftyrann', 'scrtan1': 'Rotscheiteltangare', 'semplo': 'Eskimoregenpfeifer', 'shicow': 'Seidenkuhstärling', 'sibtan2': 'Purpurtangare', 'sinwre1': 'Sinaloazaunkönig', 'sltred': 'Larvenwaldsänger', 'smbani': 'Glattschnabelani', 'snogoo': 'Schneegans', 'sobtyr1': 'Haubenkleintyrann', 'socfly1': 'Rotscheitel-Maskentyrann', 'solsan': 'Einsiedelwasserläufer', 'sonspa': 'Singammer', 'soulap1': 'Bronzekiebitz', 'sposan': 'Drosseluferläufer', 'spotow': 'Fleckengrundammer', 'spvear1': 'Glanz-Veilchenohrkolibri', 'squcuc1': 'Eichhornkuckuck', 'stbori': 'Piroltrupial', 'stejay': 'Diademhäher', 'sthant1': 'Strichelkopf-Ameisenfänger', 'sthwoo1': 'Lanzettstrichel-Baumsteiger', 'strcuc1': 'Streifenkuckuck', 'strfly1': 'Süd-Fleckenmaskentyrann', 'strsal1': 'Strichelsaltator', 'stvhum2': 'Grünamazilie', 'subfly': 'Nord-Fleckenmaskentyrann', 'sumtan': 'Sommerkardinal', 'swaspa': 'Sumpfammer', 'swathr': 'Zwergmusendrossel', 'tenwar': 'Brauenwaldsänger', 'thbeup1': 'Dickschnabelorganist', 'thbkin': 'Dickschnabel-Königstyrann', 'thswar1': 'Dreistreifen-Waldsänger', 'towsol': 'Townsendklarino', 'treswa': 'Sumpfschwalbe', 'trogna1': 'Amazonasmückenfänger', 'trokin': 'Trauerkönigstyrann', 'tromoc': 'Tropenspottdrossel', 'tropar': 'Elfenwaldsänger', 'tropew1': 'Dunkel-Waldschnäppertyrann', 'tuftit': 'Grauhäubchenmeise', 'tunswa': 'Zwergschwan', 'veery': 'Weidenmusendrossel', 'verdin': 'Goldkopf-Beutelmeise', 'vigswa': 'Veilchenschwalbe', 'warvir': 'Sängervireo', 'wbwwre1': 'Waldzaunkönig', 'webwoo1': 'Keilschnabel-Baumsteiger', 'wegspa1': 'Weißohr-Grundammer', 'wesant1': 'Kolumbienameisenwürger', 'wesblu': 'Blaukehl-Hüttensänger', 'weskin': 'Schmalschnabel-Königstyrann', 'wesmea': 'Wiesenstärling', 'westan': 'Kiefernkardinal', 'wewpew': 'Blasskehl-Schnäppertyrann', 'whbman1': 'Weißbrustpipra', 'whbnut': 'Weißbrustkleiber', 'whcpar': 'Weißstirnpapagei', 'whcsee1': 'Schwarzbrustspelzer', 'whcspa': 'Dachsammer', 'whevir': 'Weißaugenvireo', 'whfpar1': 'Weißstirnamazone', 'whimbr': 'Regenbrachvogel', 'whiwre1': 'Bartstreif-Zaunkönig', 'whtdov': 'Blauringtaube', 'whtspa': 'Weißkehlammer', 'whwbec1': 'Weißbindenbekarde', 'whwdov': 'Weißflügeltaube', 'wilfly': 'Weidenschnäppertyrann', 'willet1': 'Schlammtreter', 'wilsni1': 'Wilsonbekassine', 'wiltur': 'Truthuhn', 'wlswar': 'Mönchswaldsänger', 'wooduc': 'Brautente', 'woothr': 'Walddrossel', 'wrenti': 'Chaparralgrasmücke', 'y00475': 'Indianerblässhuhn', 'yebcha': 'Flötenstärling', 'yebela1': 'Gelbbauch-Olivtyrann', 'yebfly': 'Birkenschnäppertyrann', 'yebori1': 'Schwarzflügeltrupial', 'yebsap': 'Gelbbauch-Saftlecker', 'yebsee1': 'Gelbbauchspelzer', 'yefgra1': 'Goldbrauen-Gimpeltangare', 'yegvir': 'Gelbflankenvireo', 'yehbla': 'Brillenstärling', 'yehcar1': 'Gelbkopfkarakara', 'yelgro': 'Gelbkopf-Kernknacker', 'yelwar': 'Goldwaldsänger', 'yeofly1': 'Olivscheitel-Breitschnabeltyrann', 'yerwar': 'Kronenwaldsänger', 'yeteup1': 'Schwalbenorganist', 'yetvir': 'Gelbkehlvireo'}

In [ ]:
def normalize_list_string(string):
    return string.replace('[', '').replace(']', '').replace("'", '')

def count_normalized_list_string(string):
    string = normalize_list_string(string).strip()
    if(string == ''):
        return 0 
    return len(string.split(','))

assert(count_normalized_list_string("[]"), 0)
assert(count_normalized_list_string("['abc']"), 1)
assert(count_normalized_list_string("['abc def']"), 1)
assert(count_normalized_list_string("['abc,def']"), 2)

print("Anzahl der Arten: %s" % (
    len(train['primary_label'].value_counts())
))
print("Anzahl der Datensätze: %s" % (
    len(train)
))

train['labels_count'] = train['secondary_labels'].apply(lambda x: count_normalized_list_string(x) + 1)
print("Durchnittliche Anzahl der Arten pro Datensatz: %s" % (
    train['labels_count'].mean()
))


types = []
def unique_type_list_string(string):
    for typ in normalize_list_string(string).split(','):
        typ = typ.strip()
        if typ not in types:
            types.append(typ)

train['type'].apply(lambda x: unique_type_list_string(x))
print("Anzahl an Ruf-Typen: %s" % (len(types)))
print("Ruf-Typen: %s" % (types))

train['type_count'] = train['type'].apply(lambda x: count_normalized_list_string(x))
print("Durchschnittliche Anzahl der Ruf-Typen: %s" % (train['type_count'].mean()))

In [ ]:
base_info_data = train\
    .groupby(['primary_label', 'scientific_name', 'common_name'])\
    .agg({
        'rating': ['mean', 'min', 'max'],
        'primary_label': 'size'
    })\
    .reset_index()

base_info_data.columns = ["".join(x) for x in base_info_data.columns.ravel()]
base_info_data = base_info_data.set_index('primary_label')

# add german translation
base_info_data['german_name'] = pd.Series(german_names)

# sort columns
base_info_data = base_info_data.reindex([
    'scientific_name',
    'common_name',
    'german_name',
    'primary_labelsize',
    'ratingmean',
    'ratingmin',
    'ratingmax'
], axis=1)

base_info_data

In [ ]:
bins = [1, 100, 200, 300, 400, 500]

fig, axs = plt.subplots(figsize=(12, 4))
train.groupby('primary_label').size().value_counts(bins=bins, sort=False).plot(
    kind='bar', rot=0, ax=axs, legend=False,
    title='Verteilung Anzahl der Datensätze pro Art', 
    xlabel='Menge Datensätze',
    ylabel='Menge Arten',
    color='black',
)

In [ ]:
# Länge der einzelnen Samples
def get_length(bird, file):
    path="../input/birdclef-2021/train_short_audio/"
    audio_path=os.path.join(path,bird,file)
    return librosa.get_duration(filename=audio_path)

lengths = []
with tqdm(total=len(train)) as pbar:
    for idx, row in train.iterrows():
        pbar.update(1)
        
        lengths.append([
            row['primary_label'],
            row['filename'],
            get_length(row['primary_label'], row['filename'])
        ])

length_df = pd.DataFrame(lengths, columns=['primary_label', 'filename', 'length'])
length_df

In [ ]:
bins = [0, 5, 10, 20, 30, 40, 50, 60, 90, 120, 150, 180, 210, 240, 270, 300, 600, 1200]

fig, axs = plt.subplots(figsize=(12, 4))
length_df['length'].value_counts(bins=bins, sort=False).plot(
    kind='bar', rot=0, ax=axs, legend=False,
    title='Länge der Datensätze', 
    xlabel='Länge in Sekunden',
    ylabel='Anzahl',
    color='black',
)
length_df['length'].value_counts(bins=bins, sort=False)

In [ ]:
print("Gesamtlänge der Aufnahmen: %s Stunden" % (
    length_df['length'].sum() / 60 / 60
))

print("Durchschnittliche Länge der Audio-Dateien: %s Sekunden" % (
    length_df['length'].mean()
))

In [ ]:
# Durchschnittliche Länge der Aufnahmen pro Art hinzufügen
mean_length = length_df.groupby('primary_label').mean('length').sort_values(by='length')
base_info_data['audio_length'] = mean_length
base_info_data

In [ ]:
invalid_values = [ '', '0', '?', 'XX:XX', 'XX.XX', 'XX;XX', 'X', '??:??', '?:?', '.', 'PM', 'AM', 'NIGHT', 'DAWN' ]

# Wann wurden die Samples aufgenommen?
def convert_time(str_time):
    # None for unwanted values
    str_time = str(str_time).upper().strip()
    if(str_time in invalid_values):
        return None
    
    # make sure AM / PM is uppercase and prefixed by whitespace
    if('AM' in str_time or 'PM' in str_time):
        str_time = str_time.replace('AM', ' AM')
        str_time = str_time.replace('PM', ' PM')
        str_time = str_time.replace('  ', ' ')
        
        if(str_time.count(':') == 2):
            return time.strptime(str_time, '%H:%M:%S %p')
        return time.strptime(str_time, '%H:%M %p')

    elif(str_time.count(':') >= 1):
        first = int(str_time.split(':')[0])
        # kill leading zeros like 000:000
        if(first == 24):
            first = 0
        if first > 23:
            return None
        
        second = int(str_time.split(':')[1])
        if second > 59:
            return None
        
        return time.strptime(str(first) + ':' + str(second), '%H:%M')
    return time.strptime(str_time, '%H:%M')

def convert_time_pd(time_input):
    time_input = convert_time(time_input)
    if time_input == None:
        return None
    return time.strftime('%H:%M:%S', time_input)


train['times_cleaned'] = train['time'].apply(lambda x: convert_time_pd(x))
print('Anzahl aller Datensätze: %s' % (len(train['times_cleaned'])))

times = train['times_cleaned'].dropna()
print('Anzahl der Datensätze mit validen Zeitinformationen: %s' % (len(times)))

times = pd.DataFrame(times)
times['times_cleaned'] = pd.to_datetime(times['times_cleaned'], format="%H:%M:%S")
#print(times)

fig, axs = plt.subplots(figsize=(12, 4))

times.groupby([times['times_cleaned'].dt.hour]).agg(len).plot(
    kind='bar', rot=0, ax=axs, legend=False,
    title='Zeitverteilung der Aufnahmen', 
    xlabel='Uhrzeit',
    ylabel='Anzahl Datensätze',
    color='black',
)


In [ ]:
fig, axs = plt.subplots(figsize=(12, 4))

train['primary_label'].value_counts().plot(
    kind='bar', rot=0, ax=axs, legend=False,
    title='Verteilung Anzahl Datensätze', 
    xlabel='Vogelart',
    ylabel='Anzahl Datensätze',
    color='black',
)
axs.axes.xaxis.set_ticks([]) # remove Labels for X

In [ ]:
fig, axs = plt.subplots(figsize=(12, 4))

train['rating'].value_counts()[[5,4.5,4,3.5,3,2.5,2,1.5,1,0.5]].plot(
    kind='bar', rot=0, ax=axs, legend=False,
    title='Anzahl Datensätze nach Qualität', 
    xlabel='Bewertung',
    ylabel='Anzahl Datensätze',
    color='black',
)

In [ ]:
%matplotlib inline

# Test Audio Quality
path="./train_short_audio/"
birds=train.primary_label.unique()[:6]
file=train[train.primary_label==birds[0]]['filename'][0]


for i in range(0,2):
    file=train[train.primary_label==birds[i]]['filename'].values[0]
    audio_path=os.path.join(path,birds[i],file)
    print(birds[i])
    IPython.display.display(ipd.Audio(audio_path))
    ipd.Audio(audio_path)

In [ ]:
%%script echo skipping
# https://plotly.com/python/plotly-express/
df=train.groupby(['latitude','longitude'],as_index=False)['primary_label'].agg('count')

df=df[df.latitude!='Not specified']
fig = go.Figure()
fig.add_trace(go.Scattergeo(
        lon = df['longitude'],
        lat = df['latitude'],
        text = df['primary_label'],
        marker = dict(
            size = df['primary_label'],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        )))


fig.update_layout(
        title_text = 'Bird Samples collected From Parts of World',
        showlegend = False,
        geo = dict(
            landcolor = 'rgb(217, 217, 217)',
        )
    )

fig.show()

In [ ]:
# translate columns
columns={
    'scientific_name': 'Wissenschaftlicher Name',
    'common_name': 'Gebäuchlicher Name',
    
    'ratingmean': 'Bewertungen Durchschnitt',
    'ratingmin': 'Bewertungen Min',
    'ratingmax': 'Bewertungen Max',
    'primary_labelsize': 'Trainingsdaten Anzahl',
    'german_name': 'Deutscher Name',
    'audio_length': 'Audio-Länge Durchschnitt'
}
base_info_data = base_info_data.rename(columns=columns)


In [ ]:
path="../input/birdclef-2021/train_short_audio/"
birds=train.primary_label.unique()[:6]
file=train[train.primary_label==birds[0]]['filename'][0]

In [ ]:
plt.figure(figsize=(17,20 ))
import librosa.display

for i in range(0,6):
    file=train[train.primary_label==birds[i]]['filename'].values[0]
    audio_path=os.path.join(path,birds[i],file)
    plt.subplot(6,2,i+1)
    x, sr = librosa.load(audio_path)
    librosa.display.waveplot(x, sr=sr,color='black')
    plt.gca().set_title(birds[i])
    plt.gca().get_xaxis().set_visible(False)